# code to retrack data from old btrack versions

First, make sure you have the latest version of btrack installed:

```sh
pip install --upgrade -e git+git://github.com/quantumjot/BayesianTracker.git
```

In [3]:
import os
import napari
import btrack
import numpy as np

import csv

In [17]:
expt_list = []
with open('/home/nathan/data/kraken/h2b/giulia/control_expts.csv', newline='') as csvfile:
            control_expts_csv = csv.reader(csvfile, delimiter='\n')#, quotechar='|')
            for row in control_expts_csv:
                expt_list.append(row[0])

GV0794/Pos2
GV0794/Pos9
GV0794/Pos14
GV0795/Pos2
GV0795/Pos9
GV0795/Pos15
GV0796/Pos2
GV0796/Pos9
GV0796/Pos17
GV0796/Pos18
GV0797/Pos2
GV0797/Pos9
GV0797/Pos16
GV0797/Pos17
GV0798/Pos2
GV0798/Pos9
GV0798/Pos16
GV0798/Pos17
GV0800/Pos2
GV0800/Pos9
GV0800/Pos14
GV0801/Pos2
GV0802/Pos2
GV0804/Pos2
GV0805/Pos2
GV0806/Pos2
GV0807/Pos2
GV0801/Pos8
GV0804/Pos10
GV0804/Pos11
GV0805/Pos8
GV0806/Pos8
GV0801/Pos7
GV0804/Pos8
GV0804/Pos9
GV0805/Pos7
GV0806/Pos7
GV0801/Pos6
GV0804/Pos6
GV0804/Pos7
GV0805/Pos6
GV0806/Pos6


In [ ]:
root_path = '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/
for expt in expt_list:
    expt_ID = expt.split('/')[0]
    pos_ID = expt.split('/')[1]
    PATH = os.path.join(root_path, expt_ID, pos_ID, pos_ID+'_aligned')
    OLDFILE = os.path.join(PATH, 'segmented.hdf5')
    NEWFILE = os.path.join(PATH, 'retracked.h5')
    
    if os.path.exists(OLDFILE):
        print(OLDFILE, 'found')
    else:
        print(OLDFILE, 'NOT FOUND')

In [2]:
PATH = "/Users/arl/Downloads/"
PATH = {}/{}/{}_aligned/HDF'
OLDFILE = os.path.join(PATH, 'segmented.hdf5')
NEWFILE = os.path.join(PATH, 'retracked.h5')

In [3]:
obj_types = ['obj_type_1', 'obj_type_2']
obj_models = [
    'MDCK_config_scribble_dense.json',
    'MDCK_config_scribble_dense.json'
]

### grab the segmentation

In [4]:
with btrack.dataio.HDF5FileHandler(OLDFILE, 'r') as hdf:
    segmentation = hdf.segmentation

[INFO][2021/06/29 06:00:11 PM] Opening HDF file: /Users/arl/Downloads/segmented.hdf5...
[INFO][2021/06/29 06:00:18 PM] Loading segmentation (1194, 1200, 1600)
[INFO][2021/06/29 06:00:18 PM] Closing HDF file: /Users/arl/Downloads/segmented.hdf5


### iterate over the cell types, reorganise the data and retrack, writing out the files

In [5]:
for obj_type, obj_model in zip(obj_types, obj_models):

    with btrack.dataio.HDF5FileHandler(OLDFILE, 'r', obj_type=obj_type) as hdf:
        objects = hdf.objects
        area = hdf._hdf['objects'][obj_type]['area'][:]
        
    for idx, obj in enumerate(objects):
        obj.properties = {'area': area[idx]}
        
    # make a filtered list of objects for tracking
    filtered_objects = [o for o in objects if o.properties['area']>100.]
    
    with btrack.BayesianTracker() as tracker:

        # configure the tracker using a config file
        tracker.configure_from_file(
            os.path.join("/Users/arl/Dropbox/Code/py3/BayesianTracker/models/", obj_model)
        )
        tracker.max_search_radius = 50

        # append the objects to be tracked
        tracker.append(filtered_objects)

        # set the volume
        tracker.volume=((0, 1600), (0, 1200), (-1e5, 1e5))

        # track them (in interactive mode)
        tracker.track_interactive(step_size=100)

        # generate hypotheses and run the global optimizer
        tracker.optimize()

        tracker.export(
            NEWFILE, 
            obj_type=obj_type, 
            filter_by='area>100'
        )
        
        tracks = tracker.tracks

[INFO][2021/06/29 06:00:18 PM] Opening HDF file: /Users/arl/Downloads/segmented.hdf5...
[INFO][2021/06/29 06:00:18 PM] Loading objects/obj_type_1 (34604, 5) (34604 filtered: None)
[INFO][2021/06/29 06:00:18 PM] Closing HDF file: /Users/arl/Downloads/segmented.hdf5
[INFO][2021/06/29 06:00:18 PM] Loaded btrack: /Users/arl/Dropbox/Code/py3/BayesianTracker/btrack/libs/libtracker.dylib
[INFO][2021/06/29 06:00:18 PM] btrack (v0.4.1) library imported
[INFO][2021/06/29 06:00:18 PM] Setting max XYZ search radius to: 100
[INFO][2021/06/29 06:00:18 PM] Starting BayesianTracker session
[INFO][2021/06/29 06:00:18 PM] Loading configuration file: /Users/arl/Dropbox/Code/py3/BayesianTracker/models/MDCK_config_scribble_dense.json
[INFO][2021/06/29 06:00:18 PM] Loading motion model: b'MDCK_motion'
[INFO][2021/06/29 06:00:18 PM] Setting max XYZ search radius to: 50
[INFO][2021/06/29 06:00:18 PM] Objects are of type: <class 'list'>
[INFO][2021/06/29 06:00:18 PM] Set volume to ((0, 1600), (0, 1200), (-1000

[INFO][2021/06/29 06:00:24 PM]  - Probabilities (Link: 1.00000, Lost: 0.73882)
[INFO][2021/06/29 06:00:24 PM]  - Stats (Active: 42, Lost: 440, Conflicts resolved: 194)
[INFO][2021/06/29 06:00:24 PM] Tracking objects in frames 100 to 199 (of 1194)...
[INFO][2021/06/29 06:00:24 PM]  - Timing (Bayesian updates: 0.46ms, Linking: 0.15ms)
[INFO][2021/06/29 06:00:24 PM]  - Probabilities (Link: 1.00000, Lost: 0.61940)
[INFO][2021/06/29 06:00:24 PM]  - Stats (Active: 39, Lost: 816, Conflicts resolved: 398)
[INFO][2021/06/29 06:00:24 PM] Tracking objects in frames 200 to 299 (of 1194)...
[INFO][2021/06/29 06:00:25 PM]  - Timing (Bayesian updates: 0.68ms, Linking: 0.21ms)
[INFO][2021/06/29 06:00:25 PM]  - Probabilities (Link: 1.00000, Lost: 1.00000)
[INFO][2021/06/29 06:00:25 PM]  - Stats (Active: 54, Lost: 1223, Conflicts resolved: 580)
[INFO][2021/06/29 06:00:25 PM] Tracking objects in frames 300 to 399 (of 1194)...
[INFO][2021/06/29 06:00:25 PM]  - Timing (Bayesian updates: 1.01ms, Linking: 0.

### finally, write the segmentation back to the file

In [6]:
with btrack.dataio.HDF5FileHandler(NEWFILE, 'a') as hdf:
    hdf.write_segmentation(segmentation)
    print('btrack version: ', hdf._hdf['tracks']['obj_type_1'].attrs['version'])

[INFO][2021/06/29 06:01:02 PM] Opening HDF file: /Users/arl/Downloads/retracked.h5...
[INFO][2021/06/29 06:01:30 PM] Closing HDF file: /Users/arl/Downloads/retracked.h5


btrack version:  0.4.1


In [7]:
objects[0]

,ID,x,y,z,t,dummy,states,label,prob,area
0,0,95.124084,972.700745,0.0,0,False,0,4,0.0,411
